<a href="https://colab.research.google.com/github/PavankumarUppar/DLL/blob/main/Program8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Generate synthetic data
def generate_synthetic_data(num_samples=1000, max_sequence_length=20, embedding_dim=50):
    np.random.seed(42)
    X = np.random.randint(0, 2, size=(num_samples, max_sequence_length))
    y = np.random.randint(0, 2, size=(num_samples,))
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

# LSTM-based sentiment analysis model
class SentimentLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, embedding_dim):
        super(SentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = lstm_out[:, -1, :]
        output = self.fc(lstm_out)
        output = self.sigmoid(output)
        return output

# Hyperparameters
input_size = 2  # Binary input (0 or 1)
hidden_size = 64
num_layers = 2
output_size = 1  # Binary output (sigmoid activation)
embedding_dim = 50  # Define the embedding dimension

# Instantiate the model, loss function, and optimizer
model = SentimentLSTM(input_size, hidden_size, num_layers, output_size, embedding_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Generate synthetic data
X, y = generate_synthetic_data(num_samples=1000, max_sequence_length=20, embedding_dim=embedding_dim)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.long())
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.long())
        predictions = (outputs.squeeze() > 0.5).long()
        all_predictions.extend(predictions.tolist())

accuracy = accuracy_score(y_test.numpy(), all_predictions)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")


Accuracy on test set: 50.00%
